<a href="https://colab.research.google.com/github/Abdelrhman-Ramadan/Arabic-TTS/blob/main/Training_FastPitch_for_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading NeMo toolkit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Install dependencies
!apt-get update
# !apt-get upgrade

# !apt-get install -y build-essential python3-dev sox libsndfile1 ffmpeg
!apt-get install sox libsndfile1 ffmpeg
!pip install wget text-unidecode matplotlib>=3.3.2
# !pip install mamba-ssm
# !pip install -U pip setuptools wheel
# !pip install mamba-ssm

## Install NeMo
BRANCH = 'r2.0.0rc1'
# BRANCH = 'r2.0.0'
# BRANCH = 'main'

# !pip install "git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]" -v
!python -m pip install "git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]"


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [ ]:
# Download local version of NeMo scripts. If you are running locally and want to use your own local NeMo code,
# comment out the below lines and set NEMO_DIR to your local path.
NEMO_DIR = 'nemo'
!git clone https://github.com/NVIDIA/NeMo.git $NEMO_DIR

Cloning into 'nemo'...
remote: Enumerating objects: 188212, done.
remote: Counting objects: 100% (3496/3496), done.
remote: Compressing objects: 100% (1615/1615), done.
remote: Total 188212 (delta 2589), reused 2556 (delta 1869), pack-reused 184716 (from 1)
Receiving objects: 100% (188212/188212), 266.55 MiB | 15.03 MiB/s, done.
Resolving deltas: 100% (140944/140944), done.


# Text Preprocessing

In [ ]:
import json

def modify_json_file(file_path, output_path, old_path, new_path):
    # Load the JSON data from the file
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Parse each line as a JSON object
            try:
                entry = json.loads(line.strip())  # Use strip to remove any extra whitespace/newlines
                data.append(entry)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

    # Modify the file paths
    for entry in data:
        # Update the file path to the new directory
        entry['audio_filepath'] = entry['audio_filepath'].replace(old_path, new_path)

    # Save the modified data back to a JSON file
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)  # Use ensure_ascii=False to maintain Arabic characters
            f.write('\n')  # Write each JSON object on a new line

    print(f"File paths updated successfully for {output_path}!")

# Define paths for training and validation datasets
train_file_path = '/content/drive/MyDrive/GP_data/train_data_preprocessed.json'  # Path for training dataset
val_file_path = '/content/drive/MyDrive/GP_data/val_data_preprocessed.json'      # Path for validation dataset
train_output_path = '/content/train_data_preprocessed.json'          # Output path for modified training dataset
val_output_path = '/content/val_data_preprocessed.json'                 # Output path for modified validation dataset
old_audio_path = '/content/audio_processing'                     # Current path in the JSON
new_audio_path = '/content/drive/MyDrive/GP_data'                            # New path to replace with

# Modify JSON for training and validation
modify_json_file(train_file_path, train_output_path, old_audio_path, new_audio_path)
modify_json_file(val_file_path, val_output_path, old_audio_path, new_audio_path)


File paths updated successfully for /content/train_data_preprocessed.json!
File paths updated successfully for /content/val_data_preprocessed.json!


In [ ]:
# # Load the JSON data
# data = []
# with open('/content/train_data_preprocessed.json', 'r', encoding='utf-8') as f:
#     for line in f:
#             # Parse each line as a JSON object
#             try:
#                 entry = json.loads(line.strip())  # Use strip to remove any extra whitespace/newlines
#                 data.append(entry)
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding JSON: {e}")

# # # Shuffle the data
# # random.seed(20) # recently added this seed
# # random.shuffle(data)

# # Split the data
# split_ratio = 0.95
# split_index = int(len(data) * split_ratio)
# train_data = data[:split_index]
# val_data = data[split_index:]

# with open('/content/val_data_preprocessed_trial.json', 'w', encoding='utf-8') as f:
#         for item in val_data:
#             json.dump(item, f, ensure_ascii=False)  # Use ensure_ascii=False to maintain Arabic characters
#             f.write('\n')  # Write each JSON object on a new line



# Tokenizers

In [ ]:
# import nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers as tts_tokenizers

# class ArabicPhonemeTokenizer(tts_tokenizers.BaseTokenizer):

#     def __init__(self):
#         self.phonemes = ['$', '$$', '*', '**', '-', '<', '<<', 'A', 'AA', 'AH', 'Ah', 'D', 'DD',
#                         'E', 'EE', 'G', 'H', 'HH', 'I0', 'I0I0', 'I1', 'II0', 'J', 'S', 'SH', 'SS', 'T', 'TH',
#                         'TT', 'U0', 'U0U0', 'U1', 'UU0', 'UU1', 'Z', 'ZZ', '^', '^^', 'a', 'aa', 'b', 'bb',
#                         'd', 'dd', 'dist', 'f', 'ff', 'g', 'gg', 'h', 'hh', 'i', 'i0', 'i0i0', 'i1', 'ii0',
#                         'ii1', 'j', 'jj', 'k', 'kk', 'l', 'll', 'm', 'mm', 'n', 'nn', 'p', 'pp', 'q', 'qq',
#                         'r', 'rr', 's', 'sil', 'ss', 't', 'tt', 'u', 'u0', 'u0u0', 'u1', 'uu0', 'uu1', 'v',
#                         'w', 'ww', 'x', 'xx', 'y', 'yy', 'z', 'zz']

#         # Create phoneme dictionary
#         self.phoneme_dict = {phone: idx for idx, phone in enumerate(self.phonemes)}
#         self.vocab = self.phoneme_dict
#         self.itos = list(self.vocab.keys())  # Index-to-string mapping
#         self.stoi = self.vocab  # String-to-index mapping

#         super().__init__(tokens=self.phoneme_dict)

#     def encode(self, text):
#         # Convert a phoneme sequence into a list of indices
#         return [self.stoi[phoneme] for phoneme in text.split() if phoneme in self.stoi]

#     def decode(self, tokens):
#         # Convert a list of indices back into a phoneme sequence
#         return ' '.join([self.itos[i] for i in tokens if i < len(self.itos)])


In [ ]:
# !touch /content/arabic_phoneme_tokenizer.py

In [ ]:
# import sys
# sys.path.append('/content')  # Append the directory, not the file
# import arabic_phoneme_tokenizer

# obj1 = arabic_phoneme_tokenizer.ArabicPhonemeTokenizer()
# # obj1 = ArabicPhonemeTokenizer()

# phonemes = "y aa m u0 H a m aa d u0 < A q r a < t m aa f ii0 h aa f A q U0 l t h i0 y a f ii0 y a d i0 < a m ii0 r i0 l m u0 < m i0 n ii0 n a"
# tokens_list = list(obj1.encode(phonemes))
# print(tokens_list)
# print(obj1.pad)
# obj1.decode(tokens_list).strip() == phonemes

[89, 39, 63, 79, 16, 38, 63, 39, 42, 79, 5, 7, 69, 71, 38, 5, 76, 63, 39, 45, 55, 49, 39, 45, 7, 69, 29, 61, 76, 49, 52, 89, 38, 45, 55, 89, 38, 42, 52, 5, 38, 63, 55, 71, 52, 61, 63, 79, 5, 63, 52, 65, 55, 65, 38]
93


True

# Extracting Supplementary Data

## Loading the tokenizer

In [ ]:
!cp /content/drive/MyDrive/GP_data/scripts/arabic_phoneme_tokenizer.py /content/

## Extracting sub_data for training

/content/nemo/scripts/dataset_processing/tts/thorsten_neutral/ds_conf/ds_for_fastpitch_align.yaml

/content/nemo/scripts/dataset_processing/tts/extract_sup_data.py

### Editing the extract_sup_data.py

In [ ]:
file_path = "/content/nemo/scripts/dataset_processing/tts/extract_sup_data.py"

# Read the file contents
with open(file_path, "r") as file:
    lines = file.readlines()

# Insert the new lines at the top (or after the first import block if needed)
new_lines = ["import sys\n", "sys.path.append('/content')\n"]

# Add the new lines at the top of the file
lines = new_lines + lines

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.writelines(lines)

print(f"Added sys.path modification to {file_path}")


Added sys.path modification to /content/nemo/scripts/dataset_processing/tts/extract_sup_data.py


## Processing the train_data_preprocessed.json file sequentially in chunks of 1000 rows at a time

In [ ]:
import json
import os
import sys
import subprocess
import gc  # For garbage collection
from google.colab import files  # Import to handle file downloads

# Define paths
manifest_filepath = "/content/train_data_preprocessed.json"
output_dir = "/content/train_sup_data_chunks"
os.makedirs(output_dir, exist_ok=True)

# Process the file in chunks
chunk_size = 1000
chunk_count = 0
start_chunk = 8  # Start processing from the 8th chunk

# Open the manifest file and read it line by line to avoid loading everything in memory
with open(manifest_filepath, 'r', encoding='utf-8') as f:
    while True:
        # Initialize a list for the current chunk
        chunk = []

        # Read up to chunk_size lines from the file
        for _ in range(chunk_size):
            line = f.readline()
            if not line:
                break  # End of file reached
            try:
                entry = json.loads(line.strip())  # Parse the JSON object
                chunk.append(entry)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

        if not chunk:
            break  # No more data to process

        chunk_count += 1

        # Skip chunks until the desired start_chunk
        if chunk_count < start_chunk:
            continue

        print(f"Processing chunk {chunk_count} which size is {len(chunk)}")

        # Save the chunk to a temporary file
        chunk_manifest_path = f"{output_dir}/chunk_{chunk_count}.json"
        with open(chunk_manifest_path, "w", encoding="utf-8") as chunk_file:
            for item in chunk:
                json.dump(item, chunk_file, ensure_ascii=False)  # Ensure correct encoding
                chunk_file.write('\n')  # Write each JSON object on a new line

        # Define the command
        command = [
            "python", "/content/nemo/scripts/dataset_processing/tts/extract_sup_data.py",
            "--config-name=ds_for_fastpitch_align.yaml",
            "--config-path=/content/nemo/scripts/dataset_processing/tts/thorsten_neutral/ds_conf",
            f"manifest_filepath={chunk_manifest_path}",
            f"sup_data_path={output_dir}/sup_data_{chunk_count}",
            "++dataloader_params.num_workers=12",
            "~dataset.text_normalizer",
            "~dataset.text_normalizer_call_kwargs",
            "++dataset.text_tokenizer._target_=arabic_phoneme_tokenizer.ArabicPhonemeTokenizer",
            "~dataset.text_tokenizer.punct",
            "~dataset.text_tokenizer.apostrophe",
            "~dataset.text_tokenizer.pad_with_space"
        ]

        # Run the command and capture the output in real-time
        with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, text=True) as process:
            # Print stdout and stderr line by line in real-time
            for line in process.stdout:
                sys.stdout.write(line)
                sys.stdout.flush()  # Ensure immediate output

        # Path to the output sup_data for the current chunk
        sup_data_path = f"{output_dir}/sup_data_{chunk_count}"

        # Download the processed sup_data folder as a zip file
        zip_path = f"{sup_data_path}.zip"
        subprocess.run(["zip", "-r", zip_path, sup_data_path])  # Zip the output folder
        # files.download(zip_path)  # Download the zipped file
        # !cp $zip_path /content/drive/MyDrive/GP_data

        # Clean up after each chunk to prevent memory buildup
        del chunk
        del chunk_manifest_path
        gc.collect()  # Trigger garbage collection manually


Processing chunk 8 which size is 590
[NeMo W 2024-10-25 02:27:35 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
NumExpr defaulting to 2 threads.
PyTorch version 2.5.0+cu121 available.
Polars version 1.7.1 available.
TensorFlow version 2.17.0 available.
JAX version 0.4.33 available.
[NeMo W 2024-10-25 02:27:46 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/collections/tts/modules/common.py:206: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      @amp.autocast(False)
    
2024-10-25 02:27:47.611502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for

In [ ]:
# !zip -r /content/train_sup_data_chunks.zip /content/train_sup_data_chunks
# !cp /content/train_sup_data_chunks.zip /content/drive/MyDrive/GP_data

  adding: content/train_sup_data_chunks/ (stored 0%)
  adding: content/train_sup_data_chunks/chunk_7.json (deflated 83%)
  adding: content/train_sup_data_chunks/chunk_8.json (deflated 83%)
  adding: content/train_sup_data_chunks/chunk_5.json (deflated 83%)
  adding: content/train_sup_data_chunks/chunk_6.json (deflated 83%)


## The normal way of extract_sup_data.py

In [ ]:
!HYDRA_FULL_ERROR=1 python /content/nemo/scripts/dataset_processing/tts/extract_sup_data.py \
    --config-name="ds_for_fastpitch_align.yaml" \
    --config-path="/content/nemo/scripts/dataset_processing/tts/thorsten_neutral/ds_conf" \
    manifest_filepath=/content/train_data_preprocessed.json \
    sup_data_path=train_sup_data \
    ++dataloader_params.num_workers=12 \
    ~dataset.text_normalizer \
    ~dataset.text_normalizer_call_kwargs \
    ++dataset.text_tokenizer._target_=arabic_phoneme_tokenizer.ArabicPhonemeTokenizer \
    ~dataset.text_tokenizer.punct \
    ~dataset.text_tokenizer.apostrophe \
    ~dataset.text_tokenizer.pad_with_space \



[NeMo W 2024-10-21 16:41:23 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
NumExpr defaulting to 2 threads.
PyTorch version 2.4.1+cu121 available.
Polars version 1.7.1 available.
TensorFlow version 2.17.0 available.
JAX version 0.4.33 available.
2024-10-21 16:41:26.737031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 16:41:26.759196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 16:41:26.765643: E external/local_xla/xla/stream_

In [ ]:
# !zip -r /content/train_sup_data.zip /content/train_sup_data


  adding: content/train_sup_data/ (stored 0%)
  adding: content/train_sup_data/pitch/ (stored 0%)
  adding: content/train_sup_data/pitch/ch_08_arabic_tts_dataset_383.pt (deflated 65%)
  adding: content/train_sup_data/pitch/ch_04_arabic_tts_dataset_131.pt (deflated 62%)
  adding: content/train_sup_data/pitch/ch_19_arabic_tts_dataset_569.pt (deflated 61%)
  adding: content/train_sup_data/pitch/ch_04_arabic_tts_dataset_124.pt (deflated 60%)
  adding: content/train_sup_data/pitch/ch_18_arabic_tts_dataset_573.pt (deflated 61%)
  adding: content/train_sup_data/pitch/ch_02_arabic_tts_dataset_540.pt (deflated 63%)
  adding: content/train_sup_data/pitch/ch_07_arabic_tts_dataset_38.pt (deflated 63%)
  adding: content/train_sup_data/pitch/ch_04_arabic_tts_dataset_379.pt (deflated 64%)
  adding: content/train_sup_data/pitch/ch_20_arabic_tts_dataset_249.pt (deflated 61%)
  adding: content/train_sup_data/pitch/ch_11_arabic_tts_dataset_348.pt (deflated 63%)
  adding: content/train_sup_data/pitch/ch_1

In [ ]:
# from google.colab import files
# files.download("/content/train_sup_data.zip")


In [ ]:
!HYDRA_FULL_ERROR=1 python /content/nemo/scripts/dataset_processing/tts/extract_sup_data.py \
    --config-name="ds_for_fastpitch_align.yaml" \
    --config-path="/content/nemo/scripts/dataset_processing/tts/thorsten_neutral/ds_conf" \
    manifest_filepath=/content/drive/MyDrive/GP_data/val_data_preprocessed.json \
    sup_data_path=sup_data \
    ++dataloader_params.num_workers=4 \
    ~dataset.text_normalizer \
    ~dataset.text_normalizer_call_kwargs



[NeMo W 2024-10-16 20:13:46 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
NumExpr defaulting to 2 threads.
PyTorch version 2.4.1+cu121 available.
Polars version 1.7.1 available.
TensorFlow version 2.17.0 available.
JAX version 0.4.33 available.
2024-10-16 20:13:56.460080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 20:13:56.851812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 20:13:56.969112: E external/local_xla/xla/stream_

Processing /content/drive/MyDrive/GP_data/val_data_preprocessed.json:
100% 1898/1898 [2:02:03<00:00,  3.86s/it]
PITCH_MEAN=111.95112609863281, PITCH_STD=22.47498321533203
PITCH_MIN=65.4063949584961, PITCH_MAX=403.4817810058594

In [ ]:
# !ls /content/drive/MyDrive/GP_data/audio_preprocessed_val | grep ch_05_arabic_tts_dataset_577



# Training

In [ ]:
!pip install pyyaml


In [ ]:
!pip install wandb
!wandb login 14b38df7cc6d80879ccb75d29208bf1bc8525073

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


conf file path : /content/nemo/examples/tts/conf/de/fastpitch_align_22050_mix.yaml

conf file path :  /content/nemo/examples/tts/conf/fastpitch/fastpitch_22050.yaml

fastpich path: /content/nemo/examples/tts/fastpitch.py

/usr/local/lib/python3.10/dist-packages/omegaconf/_utils.py

/usr/local/lib/python3.10/dist-packages/nemo/collections/tts/data/dataset.py

/content/nemo/nemo/collections/tts/data/dataset.py

In [ ]:
from datetime import datetime
# import sys
# sys.path.append('/content')

In [ ]:
# Get the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"FastPitch_First_Train_Epochs_2_{current_time}"
!HYDRA_FULL_ERROR=1 CUDA_VISIBLE_DEVICES=0 python /content/nemo/examples/tts/fastpitch.py \
    --config-name="fastpitch_align_22050_mix.yaml" \
    --config-path="/content/nemo/examples/tts/conf/de" \
    model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    model.train_ds.dataloader_params.num_workers=8 \
    model.validation_ds.dataloader_params.num_workers=8 \
    model.train_ds.dataloader_params.pin_memory=True \
    model.validation_ds.dataloader_params.pin_memory=True \
    train_dataset=/content/val_data_preprocessed.json \
    validation_datasets=/content/val_data_preprocessed_trail.json \
    sup_data_path=/content/drive/MyDrive/GP_data/sup_data \
    exp_manager.exp_dir=resultArabicTTS \
    trainer.max_epochs=2 \
    trainer.check_val_every_n_epoch=1 \
    pitch_fmin=65.4063949584961 \
    pitch_fmax=403.4817810058594 \
    pitch_mean=111.95112609863281 \
    pitch_std=22.47498321533203 \
    ~model.text_normalizer \
    ~model.text_normalizer_call_kwargs \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name=$run_name \
    +exp_manager.wandb_logger_kwargs.project="ArabicTTSProject" \
    ++model.text_tokenizer._target_=arabic_phoneme_tokenizer.ArabicPhonemeTokenizer \
    ~model.text_tokenizer.g2p \
    ~model.text_tokenizer.locale \
    ~model.text_tokenizer.punct \
    ~model.text_tokenizer.apostrophe \
    ~model.text_tokenizer.pad_with_space \
    +exp_manager.checkpoint_callback_params.save_top_k=-1 \
    +exp_manager.checkpoint_callback_params.every_n_epochs=1 \
    ++exp_manager.resume_if_exists=false



2024-10-21 13:04:22.738018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 13:04:22.771514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 13:04:22.781721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 13:04:22.804798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 13:04:23.972568: W tensorflow/comp

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:22 node
      9 ?        00:00:03 oom_monitor.sh
     11 ?        00:00:00 run.sh
     12 ?        00:00:03 kernel_manager_
     41 ?        00:00:00 tail
     47 ?        00:00:00 tail
     64 ?        00:00:10 python3 <defunct>
     65 ?        00:00:15 colab-fileshim.
    114 ?        00:00:09 jupyter-noteboo
    115 ?        00:00:06 dap_multiplexer
    461 ?        00:00:33 python3
    496 ?        00:00:15 python3
    553 ?        00:00:00 bash
    554 ?        00:00:00 drive
    555 ?        00:00:00 grep
    662 ?        00:01:14 drive
    699 ?        00:00:00 fusermount <defunct>
    760 ?        00:00:00 bash
    761 ?        00:00:00 directoryprefet
    762 ?        00:00:00 tail
    763 ?        00:00:00 python3
   2710 ?        00:00:01 language_servic
   2716 ?        00:01:29 node
  30022 ?        00:00:00 sleep
  30023 ?        00:00:00 ps


In [ ]:
# !HYDRA_FULL_ERROR=1 python /content/nemo/scripts/dataset_processing/tts/extract_sup_data.py \
#     --config-name="ds_for_fastpitch_align.yaml" \
#     --config-path="/content/nemo/scripts/dataset_processing/tts/thorsten_neutral/ds_conf" \
#     manifest_filepath=/content/drive/MyDrive/GP_data/val_data_preprocessed.json \
#     sup_data_path=sup_data \
#     ++dataloader_params.num_workers=4 \
#     ~dataset.text_normalizer \
#     ~dataset.text_normalizer_call_kwargs

In [ ]:
!HYDRA_FULL_ERROR=1 python /content/nemo/examples/tts/fastpitch.py \
    --config-path="/content/nemo/examples/tts/conf/fastpitch/" \
    --config-name="fastpitch_22050.yaml" \
    train_ds_meta=/content/train_data_preprocessed.json \
    val_ds_meta=/content/val_data_preprocessed.json \
    log_ds_meta=/content/log_ds_meta.json \
    log_dir=/content/logs \
    ~vocoder_type \
    max_epochs=2 \
    n_speakers=1 \
    trainer.check_val_every_n_epoch=1 \
    pitch_fmin=65.4063949584961 \
    pitch_fmax=403.4817810058594 \
    pitch_mean=111.95112609863281 \
    pitch_std=22.47498321533203 \
    phoneme_dict_path=~ \
    heteronyms_path=~ \
    +trainer.logger=true \
    exp_manager.exp_dir=resultArabicTTS \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="FastPitch_First_Train_Epochs_2" \
    +exp_manager.wandb_logger_kwargs.project="ArabicTTSProject" \
    +model.text_tokenizer._target_=arabic_phoneme_tokenizer.ArabicPhonemeTokenizer \
    +model.text_tokenizer.g2p=~ \
    +model.text_tokenizer.locale=~ \
    +model.text_tokenizer.punct=~ \
    +model.text_tokenizer.apostrophe=~ \
    +model.text_tokenizer.pad_with_space=~ \



2024-10-19 22:26:08.469527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 22:26:08.504190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 22:26:08.514543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 22:26:08.537235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-19 22:26:09.725323: W tensorflow/compiler/tf2

In [ ]:
from nemo.collections.tts.models import HifiGanModel
HifiGanModel.list_available_models()

[NeMo W 2024-10-26 08:09:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/collections/tts/modules/common.py:206: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      @amp.autocast(False)
    


[PretrainedModelInfo(
 	pretrained_model_name=tts_en_hifigan,
 	description=This model is trained on LJSpeech audio sampled at 22050Hz and mel spectrograms generated from Tacotron2, TalkNet, and FastPitch. This model has been tested on generating female English voices with an American accent.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo,
 	class_=<class 'nemo.collections.tts.models.hifigan.HifiGanModel'>
 ),
 PretrainedModelInfo(
 	pretrained_model_name=tts_en_lj_hifigan_ft_mixertts,
 	description=This model is trained on LJSpeech audio sampled at 22050Hz and mel spectrograms generated from Mixer-TTS. This model has been tested on generating female English voices with an American accent.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_lj_hifigan/versions/1.6.0/files/tts_en_lj_hifigan_ft_mixertts.nemo,
 	class_=<class 'nemo.collections.tts.models.hifigan.HifiGanModel'>
 ),
 PretrainedModelInfo(
 	pr

In [ ]:
!ps aux | grep python


root          64  1.0  0.0      0     0 ?        Z    19:08   0:11 [python3] <defunct>
root          65  0.1  0.4  70484 54980 ?        S    19:08   0:01 python3 /usr/local/bin/colab-file
root         110  0.5  1.3 576384 178092 ?       Sl   19:08   0:05 /usr/bin/python3 /usr/local/bin/j
root         517  0.9  0.9 1139904 127812 ?      Ssl  19:09   0:09 /usr/bin/python3 -m colab_kernel_
root         552  0.2  0.1 541740 16664 ?        Sl   19:09   0:02 /usr/bin/python3 /usr/local/lib/p
root         822  0.0  0.0  17936  9456 ?        S    19:09   0:00 python3 /opt/google/drive/drive-f
root        5436  0.0  0.0   7376  3508 ?        S    19:25   0:00 /bin/bash -c ps aux | grep python
root        5438  0.0  0.0   6484  2280 ?        S    19:25   0:00 grep python


In [ ]:
# !cp -r /content/sup_data /content/drive/MyDrive/GP_data